In [1]:
import numpy as np
import cv2
import imutils
import pandas as pd
import time
import matplotlib.pyplot as plt 
import pytesseract

In [2]:
def resize_image(img):
    img = cv2.resize(img, None, fx=3, fy=3, interpolation=cv2.INTER_CUBIC)
    return img

In [3]:
def show_image(img):
    cv2.imshow("image window",img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return 'Imaged displayed successfully'

In [4]:
def detecting_licenseplate(image):
    image = imutils.resize(image, width=500)
    # cv2.imshow("Original Image", image)
    # cv2.waitKey(0)
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # cv2.imshow("1 - Grayscale Conversion", gray)

    gray = cv2.bilateralFilter(gray, 11, 17, 17)
    # cv2.imshow("2 - Bilateral Filter", gray)

    edged = cv2.Canny(gray, 170, 200)
    # cv2.imshow("4 - Canny Edges", edged)
    # ret,thresh = cv2.threshold(edged.copy(),127,255,0)
    #print(len(cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)))

    cnts,hierarchy  = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts=sorted(cnts, key = cv2.contourArea, reverse = True)[:30]
    NumberPlateCnt = None

    count = 0
    for c in cnts:
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02 * peri, True)
            if len(approx) == 4:
                NumberPlateCnt = approx
                # cv2.imshow('NumberPlateCnt',NumberPlateCnt)
                # cv2.waitKey(0)
                break

    # Masking the part other than the number plate
    mask = np.zeros(gray.shape,np.uint8)
    new_image = cv2.drawContours(mask,[NumberPlateCnt],0,255,-1)
    new_image = cv2.bitwise_and(image,image,mask=mask)
#     cv2.namedWindow("Final_image",cv2.WINDOW_NORMAL)
    #croping numberplate 
#     show_image(new_image)
#     new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
#     new_image = cv2.threshold(new_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    img_numberplate = np.where(new_image>0.8)
    last_index, = img_numberplate[0].shape
    x1,y1 = img_numberplate[1].min(),img_numberplate[0].min()
    x2,y2 = img_numberplate[1].max(),img_numberplate[0].max()
#     print("x1 = "+str(x1)+"x2 = "+str(x2)+"y1 = "+str(y1)+"y2 = "+str(y2))
    new_image = new_image[y1:y2,x1:x2]
#     show_image(new_image)
    return new_image

In [5]:
def image_cleaning(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.erode(img, kernel, iterations=500)
    img = cv2.dilate(img, kernel, iterations=1)
    img = cv2.GaussianBlur(img, (3, 3), 1)
    img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    return img

In [6]:
def reading_licenseplate(img):
    config = ('-l eng --oem 1 --psm 3')
    license_number = pytesseract.image_to_string(img, config=config)
    return license_number

In [7]:
def saving_image(img,name):
    cv2.imwrite(name,img)
    print('Imaged saved.')
    return True

In [8]:
# C:\Users\KARAN KALRA\PycharmProjects\FirstProject\Yolo\alot_number.jpg

In [11]:
path_of_image = 'G:/LPR Project/Labled data/34.jpeg'
license_number = ""
try:
    img = cv2.imread(path_of_image)
#         show_image(img)
    img = detecting_licenseplate(img)
#     plt.imshow(img)
#         print("shape : "+str(img.shape))
    img = resize_image(img)
    img = image_cleaning(img)
    license_number = reading_licenseplate(img)
#         arr[x] = license_number
except Exception as err:
    pass

print(license_number)

MH 20 E7597
